In [1]:
# !jupyter serverextension enable voila --sys-prefix
# !voila --enable_nbextensions=True

In [2]:
from ipywidgets import widgets, VBox
from PIL import ImageGrab
import pyautogui
import win32gui
import time
import numpy as np
from IPython.display import clear_output
import re
import requests
import lxml.html as lh
import asyncio

In [3]:
tables = {}

In [4]:
class HaltableExecutor:

    def __init__(self, togglebutton):
        self._togglebutton = togglebutton

    async def my_code(self):
        # This is your user code
        while True:
            get_tables()
            if self._togglebutton.value:
                drop_table.disabled = True
                btn_tables.disabled = True
                
                if active_table_no in tables:
                    active_table = tables[active_table_no]
                    lbl_table.value = active_table

                    blinds = list(map(int, re.search(r'\d+\/\d+', active_table).group().split('/')))
                    lbl_blinds.value = 'Blinds are %d/%d' %(blinds[0], blinds[1])
                    lbl_player1.value = 'Player 1 stack %d' % 600
                    lbl_player2.value = 'Player 2 stack %d' % 600
                    lbl_player3.value = 'Player 3 stack %d' % 600
                    lbl_button.value = 'Player %d is the button' % 1
                
                else: btn_on_off.value = False
                
            else:
                drop_table.disabled = False
                btn_tables.disabled = False
                lbl_blinds.value = ''
                lbl_player1.value = ''
                lbl_player2.value = ''
                lbl_player3.value = ''
                lbl_button.value = ''
                
                if len(tables) > 0:
                    active_table_no = list(tables.keys())[list(tables.values()).index(drop_table.value)]
                
            await asyncio.sleep(0.1)  # use this to temporarily give up control of the event loop to allow scheduling checkbox changes
        
    def start(self):
        print('starting')
        asyncio.ensure_future(self.my_code())

In [5]:
def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))


def get_tables():
    tables.clear()
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "tournament" and "table" in i[1].lower():
            tables[i[0]] = i[1]
    
    if len(tables) > 0:
        drop_table.options = list(tables.values())
    
    else: drop_table.options = ['No open tables']
    
def get_table(table_id=None):
    tables.clear()
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "tournament" and "table" in i[1].lower():
            tables[i[1]] = i[0]
        drop_table.options = list(tables.keys())
        
            
    if table_id == None:
        if len(tables) == 0:
            lbl_table.value = 'No tables!'
            return False
        else: 
            return tables
    
    else:
        for table in tables:
            if table[0] == table_id:
                return table[1]
        
        print('Table no longer available')
        return False

In [6]:
btn_on_off = widgets.ToggleButton(description='Tracking On/Off')

In [7]:
drop_table = widgets.Dropdown(options=[], 
                              value=None, # can use tuples of table_no, table_names here i fyou want
                              description='Select Table:',)

In [8]:
btn_tables = widgets.Button(description='Refresh Tables')

In [9]:
lbl_table = widgets.Label()
lbl_blinds = widgets.Label()
lbl_player1 = widgets.Label()
lbl_player2 = widgets.Label()
lbl_player3 = widgets.Label()
lbl_button = widgets.Label()

In [10]:
def on_button_clicked(click):
    get_tables()

In [11]:
def on_select(change):
    lbl_table.value = drop_table.value

In [12]:
btn_tables.on_click(on_button_clicked)
drop_table.observe(on_select, 'value')

In [13]:
get_tables()

In [14]:
display(VBox([btn_on_off, 
              btn_tables, 
              widgets.HTML('<br>'), 
              drop_table, 
              widgets.HTML('<br>'), 
              lbl_table,
              lbl_blinds,
              lbl_player1,
              lbl_player2,
              lbl_player3,
              lbl_button]))

Task exception was never retrieved
future: <Task finished coro=<HaltableExecutor.my_code() done, defined at <ipython-input-4-d411d4c8dc3f>:6> exception=UnboundLocalError("local variable 'active_table_no' referenced before assignment")>
Traceback (most recent call last):
  File "<ipython-input-4-d411d4c8dc3f>", line 14, in my_code
    if active_table_no in tables:
UnboundLocalError: local variable 'active_table_no' referenced before assignment


In [15]:
exe = HaltableExecutor(btn_on_off)
exe.start()

starting
